In [135]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import math

Q1: Parametric Classification: Using each of the 64 input features separately as the single input dimension, use parametric classification, assuming that the input is distributed according to a Gaussian. Report the training and test errors for the case of each of the 64 features. Which feature(s) give the best test performance? 

In [136]:

train_data = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69_N200.tra', sep=",")
train_data

,0,0.1,5,14,4,0.2,0.3,0.4,0.5,0.6,...,0.27,0.28,0.29,4.6,12.1,14.4,7,0.30,0.31,6.1
0,0,0,0,10,12,0,0,0,0,0,...,0,0,0,1,11,14,12,1,0,6
1,0,0,2,14,10,0,0,0,0,1,...,0,0,0,1,12,14,12,4,0,6
2,0,0,2,13,5,0,0,0,0,0,...,0,0,0,4,15,12,9,1,0,6
3,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
4,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,8,13,10,5,0,0,0,...,0,0,0,0,14,15,2,0,0,9
196,0,0,0,3,10,13,12,4,0,0,...,0,0,0,0,1,15,1,0,0,9
197,0,0,0,4,10,13,12,0,0,0,...,0,0,0,0,2,11,0,0,0,9
198,0,0,6,14,16,9,0,0,0,5,...,0,0,0,8,14,16,13,1,0,9


In [137]:
# use train_array as numpy array

train_array = np.asarray(train_data)
print(train_array)
[train_rows, train_col] = train_array.shape

# now convert split the arrays into two parts as we have two classes here
train_class6 = train_array[train_array[:,64] == 6, :]
print(train_class6)

train_class9 = train_array[train_array[:,64] == 9, :]
print(train_class9)

[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]
[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 7 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 1 ... 3 0 6]]
[[0 0 0 ... 0 0 9]
 [0 0 1 ... 0 0 9]
 [0 0 0 ... 0 0 9]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]


In [138]:
# get the occurances or count of both the classes

[train_class6_rows, train_col] = train_class6.shape
print("Number of rows of class 6 of training dataset : ", train_class6_rows)

[train_class9_rows, train_col] = train_class9.shape
print("Number of rows of class 9 of training dataset : ",train_class9_rows)


Number of rows of class 6 of training dataset :  100
Number of rows of class 9 of training dataset :  100


In [139]:
# now will fine out the class priors

train_prior_class6 = train_class6_rows/train_rows
print("Prior probability of class 6 of training data : ", train_prior_class6)

train_prior_class9 = train_class9_rows/train_rows
print("Prior probability of class 9 of training data : ", train_prior_class9)


Prior probability of class 6 of training data :  0.5
Prior probability of class 9 of training data :  0.5


In [140]:
# now find the mean and std deviation for each feature of the classes (6 and 9). We have 64 features here and 2 classes.

train_df = pd.DataFrame(columns = ["Train_Mean_class6", "Train_Std_Deviation_class6", "Train_Mean_class9", "Train_Std_Deviation_class9"])

for icol in range(train_col-1):
    train_mean_class6 = np.mean(train_class6[:,icol])
    train_std_dv_class6 = np.std(train_class6[:,icol])

    train_mean_class9 = np.mean(train_class9[:,icol])
    train_std_dv_class9 = np.std(train_class9[:,icol])
    
    train_df = train_df.append({'Train_Mean_class6': train_mean_class6, 'Train_Std_Deviation_class6': train_std_dv_class6, 'Train_Mean_class9': train_mean_class9, 'Train_Std_Deviation_class9': train_std_dv_class9}, ignore_index=True)


print(train_df)



    Train_Mean_class6  Train_Std_Deviation_class6  Train_Mean_class9  \
0                0.00                    0.000000               0.00   
1                0.00                    0.000000               0.10   
2                1.59                    2.898603               4.32   
3               10.38                    3.405231              10.99   
4                8.19                    5.266299              11.44   
..                ...                         ...                ...   
59              10.01                    2.988294              11.35   
60              14.62                    1.369525              12.42   
61              13.60                    2.619160               7.45   
62               6.50                    4.332436               1.61   
63               0.29                    1.070467               0.05   

    Train_Std_Deviation_class9  
0                     0.000000  
1                     0.331662  
2                     3.627341  
3  

In [153]:

# for class 6 find error of each feature

# train_feature_errors_class6 = []
train_feature_errors_class6_dict = {}

for icol in range(train_col-1):
    train_mean_class6 = train_df.Train_Mean_class6[icol]
    train_std_class6 = train_df.Train_Std_Deviation_class6[icol]
    train_mean_class9 = train_df.Train_Mean_class9[icol]
    train_std_class9 = train_df.Train_Std_Deviation_class9[icol]
    train_error_of_feature = 0
    for irow in range (train_rows):
        if (~(train_std_class6 == 0.0) & ~(train_std_class9 == 0.0)):
            train_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class6) - ((train_array[irow,icol] - train_mean_class6)** 2)/(2*(train_std_class6**2)) + math.log(train_prior_class6)
            train_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class9) - ((train_array[irow,icol] - train_mean_class9)** 2)/(2*(train_std_class9**2)) + math.log(train_prior_class6)
            if (train_g_class6 > train_g_class9):
                if ~(train_array[irow, 64] == 6):
                    train_error_of_feature = train_error_of_feature + 1
            else:
                if ~(train_array[irow, 64] == 9):
                    train_error_of_feature = train_error_of_feature + 1
    if (train_error_of_feature != 0):
        train_feature_errors_class6_dict.update({"feature" + str(icol+1) :train_error_of_feature })
    
print("-------------printing list of training errors for each feature-----------------" + " \n")    
print(train_feature_errors_class6_dict)
print("\n")
print("sorted : "+"\n")
train_feature_errors_class6_sortedDict = sorted(train_feature_errors_class6_dict.items(), key=lambda item: item[1], reverse = False)
print(train_feature_errors_class6_sortedDict)



-------------printing list of training errors for each feature----------------- 

{'feature3': 62, 'feature4': 64, 'feature5': 68, 'feature6': 63, 'feature10': 62, 'feature11': 59, 'feature12': 77, 'feature13': 59, 'feature14': 20, 'feature18': 54, 'feature19': 91, 'feature20': 76, 'feature21': 37, 'feature26': 93, 'feature27': 78, 'feature28': 73, 'feature29': 30, 'feature30': 4, 'feature31': 19, 'feature34': 44, 'feature35': 19, 'feature36': 50, 'feature37': 57, 'feature38': 74, 'feature39': 72, 'feature42': 56, 'feature43': 1, 'feature44': 18, 'feature45': 89, 'feature46': 90, 'feature47': 54, 'feature50': 98, 'feature51': 45, 'feature52': 39, 'feature53': 74, 'feature54': 107, 'feature55': 35, 'feature56': 80, 'feature58': 93, 'feature59': 62, 'feature60': 53, 'feature61': 72, 'feature62': 48, 'feature63': 50, 'feature64': 88}


sorted : 

[('feature43', 1), ('feature30', 4), ('feature44', 18), ('feature31', 19), ('feature35', 19), ('feature14', 20), ('feature29', 30), ('feature55'

In [142]:
# now we have to repeat all procedures mentioned above for test data

test_data = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69.tes', sep=",")
test_data


,0,0.1,0.2,12,13,0.3,0.4,0.5,0.6,0.7,...,0.31,0.32,0.33,1,9,15.2,11.1,3.2,0.34,6
0,0,0,0,8,15,1,0,0,0,0,...,2,0,0,0,7,15,16,11,0,6
1,0,0,1,13,14,3,0,0,0,0,...,0,0,0,3,12,15,14,7,0,6
2,0,0,0,5,14,2,0,0,0,0,...,0,0,0,0,5,16,15,3,0,6
3,0,0,0,10,13,1,0,0,0,1,...,0,0,0,0,7,15,15,5,0,6
4,0,0,0,9,10,0,0,0,0,0,...,0,0,0,0,6,12,14,9,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,7,16,9,8,2,0,0,5,...,0,0,0,7,15,14,7,0,0,9
356,0,0,5,14,11,8,0,0,0,4,...,0,0,0,7,15,12,5,0,0,9
357,0,0,4,14,15,6,0,0,0,5,...,0,0,0,4,13,15,9,0,0,9
358,0,0,4,10,13,6,0,0,0,1,...,0,0,0,2,14,15,9,0,0,9


In [143]:
# use train_array as numpy array

test_array = np.asarray(test_data)
print(test_array)
[test_rows, test_col] = test_array.shape

# now convert split the arrays into two parts as we have two classes here

test_class6 = test_array[test_array[:,64] == 6, :]
print(test_class6)

test_class9 = test_array[test_array[:,64] == 9, :]
print(test_class9)

[[ 0  0  0 ... 11  0  6]
 [ 0  0  1 ...  7  0  6]
 [ 0  0  0 ...  3  0  6]
 ...
 [ 0  0  4 ...  0  0  9]
 [ 0  0  4 ...  0  0  9]
 [ 0  0  2 ...  0  0  9]]
[[ 0  0  0 ... 11  0  6]
 [ 0  0  1 ...  7  0  6]
 [ 0  0  0 ...  3  0  6]
 ...
 [ 0  0  2 ...  7  0  6]
 [ 0  0  0 ...  5  0  6]
 [ 0  0  2 ...  4  0  6]]
[[ 0  0 11 ...  0  0  9]
 [ 0  0  6 ... 11  1  9]
 [ 0  0  9 ... 11  0  9]
 ...
 [ 0  0  4 ...  0  0  9]
 [ 0  0  4 ...  0  0  9]
 [ 0  0  2 ...  0  0  9]]


In [144]:
# get the occurances or count of both the classes

[test_class6_rows, test_col] = test_class6.shape
print("Number of rows of class 6 of testing dataset : ", test_class6_rows)

[test_class9_rows, test_col] = test_class9.shape
print("Number of rows of class 9 of testing dataset : ", test_class9_rows)



Number of rows of class 6 of testing dataset :  180
Number of rows of class 9 of testing dataset :  180


In [145]:
# now will fine out the class priors

test_prior_class6 = test_class6_rows/test_rows
print("Prior probability of class 6 of testing data : ", test_prior_class6)

test_prior_class9 = test_class9_rows/test_rows
print("Prior probability of class 9 of testing data : ", test_prior_class9)


Prior probability of class 6 of testing data :  0.5
Prior probability of class 9 of testing data :  0.5


In [152]:
# calculate test errors
test_feature_errors_class6_dict = {}

for icol in range(test_col-1):
    train_mean_class6 = train_df.Train_Mean_class6[icol]
    train_std_class6 = train_df.Train_Std_Deviation_class6[icol]
    train_mean_class9 = train_df.Train_Mean_class9[icol]
    train_std_class9 = train_df.Train_Std_Deviation_class9[icol]
    test_error_of_feature = 0
    for irow in range (test_rows):
        if (~(train_std_class6 == 0.0) & ~(train_std_class9 == 0.0)):
            test_g_class6 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class6) - ((test_array[irow,icol] - train_mean_class6)** 2)/(2*(train_std_class6**2)) + math.log(test_prior_class6)
            test_g_class9 = -((1/2)* math.log(2*math.pi)) - math.log(train_std_class9) - ((test_array[irow,icol] - train_mean_class9)** 2)/(2*(train_std_class9**2)) + math.log(test_prior_class6)
            if (test_g_class6 > test_g_class9):
                if ~(test_array[irow, 64] == 6):
                    test_error_of_feature = test_error_of_feature + 1
            else:
                if ~(test_array[irow, 64] == 9):
                    test_error_of_feature = test_error_of_feature + 1
    if (test_error_of_feature != 0):
        test_feature_errors_class6_dict.update({"feature" + str(icol+1) :test_error_of_feature })
    
print("-------------printing list of testing errors for each feature-----------------" + " \n")        
print(test_feature_errors_class6_dict)
print("\n")
print("sorted : "+"\n")
test_feature_errors_class6_sortedDict = sorted(test_feature_errors_class6_dict.items(), key=lambda item: item[1], reverse = False)
print(test_feature_errors_class6_sortedDict)

-------------printing list of testing errors for each feature----------------- 

{'feature3': 80, 'feature4': 136, 'feature5': 151, 'feature6': 109, 'feature10': 93, 'feature11': 81, 'feature12': 85, 'feature13': 128, 'feature14': 29, 'feature18': 108, 'feature19': 185, 'feature20': 129, 'feature21': 54, 'feature26': 197, 'feature27': 124, 'feature28': 118, 'feature29': 57, 'feature30': 29, 'feature31': 48, 'feature34': 68, 'feature35': 30, 'feature36': 74, 'feature37': 89, 'feature38': 163, 'feature39': 134, 'feature42': 83, 'feature43': 4, 'feature44': 33, 'feature45': 136, 'feature46': 184, 'feature47': 123, 'feature50': 163, 'feature51': 101, 'feature52': 60, 'feature53': 165, 'feature54': 172, 'feature55': 95, 'feature56': 162, 'feature58': 166, 'feature59': 83, 'feature60': 120, 'feature61': 147, 'feature62': 107, 'feature63': 132, 'feature64': 177}


sorted : 

[('feature43', 4), ('feature14', 29), ('feature30', 29), ('feature35', 30), ('feature44', 33), ('feature31', 48), ('fea

Answer: For the given testing data set, we get the error for given features as shown above. The top 10 features who give the best performance among all are as :
feature43, feature14, feature30, feature35, feature44, feature31, feature21, feature29, feature52, feature34.



Q2: Use all the 64 features, assume that inputs are 64 dimensional Gaussians, and assume that for each class the covariance matrix is different. Report the training and test confusion matrices and errors. Hint: eliminate features that have variance zero. 

Q3: Repeat Q2, assuming that all the class covariance matrices are the same. 

Q4: Use the first 10 features in Q1 that gave the best test performance and repeat Q2. Compare the test error performance you got to Q2. 